### SPARK SESSION AND DATA LOADING

In [1]:
#Apache Spark Libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import input_file_name, regexp_extract, regexp_replace, split, col

## Spark session
# Set configs
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("batch_pipeline")
sparkConf.set("spark.driver.memory", "4g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# Start spark session
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

## Load
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Get bucket path
gcs_bucket_path = "bucket_path"  # Set  bucket path

# Set file path
csv_files_path = gcs_bucket_path + "*.csv"

# Load Spark DF
df = spark.read.csv(csv_files_path, header=True, inferSchema=True)

print(f"Number of rows: {df.count()}")



Number of rows: 4077352


### DATA EXTRACTION

In [2]:
## Extract the year from the filename (handles both YYYY and YYYY-MM-DD)
# Add a column with the file name
df = df.withColumn("filename", regexp_replace(input_file_name(), r"bucket_path", ""))   # BUCKET PATH!!

# Extract the year from the filename (handles both YYYY and YYYYMMDD patterns)

df = df.withColumn(
    "year",
    regexp_extract(col("filename"), r".*_(?:\d{2}\d{2})?(\d{4})\.csv", 1).cast("int")
)

# Extract net manager and type
df = df.withColumn('leverancier', split(col("filename"), '_').getItem(0))


# Check
df.printSchema()
df.show()

root
 |-- net_manager: string (nullable = true)
 |-- purchase_area: string (nullable = true)
 |-- street: string (nullable = true)
 |-- zipcode_from: string (nullable = true)
 |-- zipcode_to: string (nullable = true)
 |-- city: string (nullable = true)
 |-- num_connections: string (nullable = true)
 |-- delivery_perc: string (nullable = true)
 |-- perc_of_active_connections: string (nullable = true)
 |-- type_conn_perc: string (nullable = true)
 |-- type_of_connection: string (nullable = true)
 |-- annual_consume: string (nullable = true)
 |-- annual_consume_lowtarif_perc: string (nullable = true)
 |-- smartmeter_perc: string (nullable = true)
 |-- filename: string (nullable = false)
 |-- year: integer (nullable = true)
 |-- leverancier: string (nullable = true)

+-----------+-------------+--------------------+------------+----------+---------+---------------+-------------+--------------------------+--------------+------------------+--------------+----------------------------+---------

Extract the aggregate consumption data from the dataset

In [3]:
from pyspark.sql.functions import sum as _sum, col, round

# Perform group by and aggregate
aggregated_df = df.groupBy("zipcode_from", "zipcode_to", "city","year","leverancier").agg(
    round(_sum("annual_consume"), 2).alias("annual_consumption"),
    _sum("num_connections").alias("total_num_connections")
)

# Calculate average consumption per connection
aggregated_df = aggregated_df.withColumn(
    "average_consumption",
    round(col("annual_consumption") / col("total_num_connections"), 2)
)

# Sort by average consumption in descending order
aggregated_df = aggregated_df.orderBy(col("average_consumption").desc())

# Show the result
aggregated_df.show()

+------------+----------+-------------+----+-----------+------------------+---------------------+-------------------+
|zipcode_from|zipcode_to|         city|year|leverancier|annual_consumption|total_num_connections|average_consumption|
+------------+----------+-------------+----+-----------+------------------+---------------------+-------------------+
|      5631BL|    5631BM|    EINDHOVEN|2013|    endinet|          429931.2|                 10.0|           42993.12|
|      5631BL|    5631BM|    EINDHOVEN|2012|    endinet|          429881.8|                 10.0|           42988.18|
|      5631BL|    5631BM|    EINDHOVEN|2011|    endinet|          428277.4|                 10.0|           42827.74|
|      5622PG|    5622PH|    EINDHOVEN|2012|    endinet|         290233.53|                 17.0|           17072.56|
|      5611KS|    5611KT|    EINDHOVEN|2012|    endinet|          157011.4|                 10.0|           15701.14|
|      5633AC|    5633AE|    EINDHOVEN|2013|    endinet|

In [4]:
# Remove any outliers from the dataset
from pyspark.sql.functions import col, expr, lit, percentile_approx

# Step 1: Calculate Q1 and Q3
percentiles = aggregated_df.select(
    percentile_approx(col("average_consumption"), 0.25, lit(100)).alias("Q1"),
    percentile_approx(col("average_consumption"), 0.75, lit(100)).alias("Q3")
).collect()[0]

Q1, Q3 = percentiles.Q1, percentiles.Q3
IQR = Q3 - Q1

# Step 2: Calculate bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 3: Filter out outliers
aggregated_df = aggregated_df.filter(
    (col("average_consumption") >= lower_bound) & (col("average_consumption") <= upper_bound)
)

# Show the result
print(f"Number of rows: {aggregated_df.count()}")


Number of rows: 3809752


### QUERIES

What is the energy consumption each year?

In [5]:
from pyspark.sql.functions import sum as _sum, col, round

# Sum annual consumption, grouped by year
yearly_consumption = aggregated_df.groupBy("year").agg(
    _sum("annual_consumption").alias("yearly_consumption")
)

# Cast to decimal and round
yearly_consumption = yearly_consumption.withColumn(
    "yearly_consumption",
    round(col("yearly_consumption"), 2)
)

yearly_consumption = yearly_consumption.orderBy(col("year").asc())

yearly_consumption.show()

+----+------------------+
|year|yearly_consumption|
+----+------------------+
|2009|      7.44615414E8|
|2010|     1.280313545E9|
|2011|   1.30864941953E9|
|2012|   1.32543587561E9|
|2013|   1.37647144216E9|
|2014|     1.363139194E9|
|2015|     1.331306962E9|
|2016|     1.322117553E9|
|2017|     1.311258097E9|
|2018|   1.18936275021E9|
|2019|   1.16539894699E9|
|2020|   1.12582158877E9|
+----+------------------+



How much energy do specific zipcodes use yearly?

In [6]:
from pyspark.sql.functions import sum as _sum, col, round

# Group by supplier
zipcode_total = aggregated_df.groupBy("zipcode_from", "zipcode_to","year").agg(
    _sum("annual_consumption").alias("total_consumption")
)

zipcode_total.show()

+------------+----------+----+-----------------+
|zipcode_from|zipcode_to|year|total_consumption|
+------------+----------+----+-----------------+
|      2622BV|    2622BV|2013|           4598.0|
|      1185TW|    1185TW|2014|           7353.0|
|      8463TK|    8463VA|2020|          11948.0|
|      4463AK|    4463AL|2018|           6431.0|
|      9285WN|    9285WN|2011|           5053.0|
|      8121ER|    8121ER|2020|           5050.0|
|      3911MG|    3911MG|2012|           9171.0|
|      2665RW|    2665RW|2010|           5041.0|
|      1435GV|    1435GV|2014|           5037.0|
|      6005AD|    6005AD|2010|           7323.0|
|      3401NH|    3401NJ|2019|          10069.0|
|      4811TP|    4811TP|2019|           9153.1|
|      6645AJ|    6645AJ|2013|           8695.0|
|      1017CE|    1017CE|2012|          10061.0|
|      4454PR|    4454PS|2015|           5935.0|
|      2242SZ|    2242SZ|2009|           5476.0|
|      4116DC|    4116DC|2011|           4562.0|
|      8181ME|    81

How much energy is supplier by suppliers on a yearly basis?

In [8]:
from pyspark.sql.functions import sum as _sum, col, round

# Group by supplier
supplier_total = aggregated_df.groupBy("leverancier", "year").agg(
    _sum("annual_consumption").alias("total_consumption")
)

supplier_total.show()

+------------------+----+--------------------+
|       leverancier|year|   total_consumption|
+------------------+----+--------------------+
|             rendo|2013|           5036136.0|
|            stedin|2020|2.5886791611999595E8|
|    westland-infra|2019|         1.4097802E7|
|    westland-infra|2018|         1.2189676E7|
|            enexis|2013|        5.53952919E8|
|           liander|2014|        4.74047378E8|
|           liander|2009|         4.6740882E8|
|           liander|2016|        4.73964545E8|
|            enexis|2016|        5.04151692E8|
|enduriselectricity|2013|         3.1346585E7|
|           liander|2011|        4.77433164E8|
|            enexis|2020| 3.834581936499889E8|
|             coteq|2019|           8115602.0|
|            stedin|2014|        2.80646724E8|
|           liander|2019|        4.62511608E8|
|            stedin|2009|        2.77206594E8|
|             rendo|2018|           4589365.0|
|            stedin|2016|        2.73919385E8|
|            

### DATA EXPORT

In [9]:
# Save table to BQ
# Use the Cloud Storage bucket for temporary BigQuery export data the connector uses.
bucket = "your_temp_bucket"
spark.conf.set('temporaryGcsBucket', bucket)

# Energy consumption per year
yearly_consumption.write.format('bigquery') \
  .option('table', '') \ # Set table name
  .mode("overwrite") \
  .save()


# Energy consumption per zip code
zipcode_total.write.format('bigquery') \
  .option('table', '') \# Set table name
  .mode("overwrite") \
  .save()


# Energy consumption per supplier
supplier_total.write.format('bigquery') \
  .option('table', '') \# Set table name
  .mode("overwrite") \
  .save()


In [10]:
# Stop session
spark.stop()